# Pandas (continues)

In [1]:
import pandas as pd
import numpy as np

## Creation of dataframes

The DataFrame is essentially a two dimensional object, and it can be created in three different ways:

* out of a two dimensional NumPy array
* out of given columns
* out of given rows

### Creating DataFrames from a NumPy array

In the following example a DataFrame with 2 rows and 3 column is created. The row and column indices are given explicitly.

In [2]:
df=pd.DataFrame(np.random.randn(2,3), columns=["First", "Second", "Third"], index=["a", "b"])
df

,First,Second,Third
a,1.273012,-1.645268,0.133877
b,0.742194,-0.225893,2.600842


Note that now both the rows and columns can be accessed using the special `Index` object:

In [3]:
df.index                            # These are the "row names"

Index(['a', 'b'], dtype='object')

In [4]:
df.columns                          # These are the "column names"

Index(['First', 'Second', 'Third'], dtype='object')

If either `columns` or `index` argument is left out, then an implicit integer index will be used:

In [5]:
df2=pd.DataFrame(np.random.randn(2,3), index=["a", "b"])
df2

,0,1,2
a,0.095937,-0.688698,0.653831
b,0.978782,0.034069,1.125257


Now the column index is an object similar to Python's builtin `range` type:

In [6]:
df2.columns

RangeIndex(start=0, stop=3, step=1)

### Creating DataFrames from columns

A column can be specified as a list, an NumPy array, or a Pandas' Series. The names of the columns can be given either with the `columns` parameter, or if Series objects are used, then the `name` attribute of each Series is used as the column name.

In [7]:
s1 = pd.Series([1,2,3])
s1

0    1
1    2
2    3
dtype: int64

In [8]:
s2 = pd.Series([4,5,6], name="b")
s2

0    4
1    5
2    6
Name: b, dtype: int64

Give the column name explicitly:

In [9]:
pd.DataFrame(s1, columns=["a"])

,a
0,1
1,2
2,3


Use the `name` attribute of Series s2 as the column name:

In [10]:
pd.DataFrame(s2)

,b
0,4
1,5
2,6


If using multiple columns, then they must be given as the dictionary, whose keys give the column names and values are the actual column content.

In [11]:
pd.DataFrame({"a": s1, "b": s2})

,a,b
0,1,4
1,2,5
2,3,6


### Creating DataFrames from rows

We can give a list of rows as a parameter to the DataFrame constructor. Each row is given as a dict, list, Series, or NumPy array. If we want to give names for the columns, then either the rows must be dictionaries, where the key is the column name and the values are the elements of the DataFrame on that row and column, or else the column names must be given explicitly. An example of this:

In [12]:
df=pd.DataFrame([{"Wage" : 1000, "Name" : "Jack", "Age" : 21}, {"Wage" : 1500, "Name" : "John", "Age" : 29}])
df

,Age,Name,Wage
0,21,Jack,1000
1,29,John,1500


Or:

In [13]:
df = pd.DataFrame([[1000, "Jack", 21], [1500, "John", 29]], columns=["Wage", "Name", "Age"])
df

,Wage,Name,Age
0,1000,Jack,21
1,1500,John,29


<div class="alert alert-warning">Note that the order of columns is not always the same order as they were in the parameter list. In this case you can use the `columns` parameter to specify the exact order.


In the earlier case, however, where we created DataFrames from a dictionary of columns, the order of columns should be the same as in the parameter dictionary in the recent versions of Python and Pandas.
</div>

In the sense of information content the order of columns should not matter, but sometimes you want to specify a certain order to make the Frame more readable, or to make it obey some semantic meaning of column order.

## Accessing columns and rows of a dataframe

Even though DataFrames are basically just two dimensional arrays, the way to access their elements is different from NumPy arrays. There are a couple of complications, which we will go through in this section.

Firstly, the bracket notation `[]` does not allow the use of an index pair to access a single element of the DataFrame. Instead only one dimension can be specified.

Well, does this dimension specify the rows of the DataFrame, like NumPy arrays if only one index is given, or does it specify the columns of the DataFrame?

It depends!

If an integer is used, then it specifies a column of the DataFrame in the case the **explicit** indices for the column contain that integer. In any other case an error will result. For example, with the above DataFrame, the following indexing will not work, because the explicit column index consist of the column names "Name" and "Wage" which are not integers.

In [14]:
try:
    df[0]
except KeyError:
    import sys
    print("Key error", file=sys.stderr)

Key error


The following will however work.

In [15]:
df["Wage"]

0    1000
1    1500
Name: Wage, dtype: int64

As does the fancy indexing:

In [16]:
df[["Wage", "Name"]]

,Wage,Name
0,1000,Jack
1,1500,John


If one indexes with a slice or a boolean mask, then the **rows** are referred to. Examples of these:

In [17]:
df[0:1]                           # slice

,Wage,Name,Age
0,1000,Jack,21


In [18]:
df[df.Wage > 1200]               # boolean mask

,Wage,Name,Age
1,1500,John,29


If some of the above calls return a Series object, then you can chain the bracket calls to get a single value from the DataFrame:

In [19]:
df["Wage"][1]                    # Note order of dimensions

1500

But there is a better way to achieve this, which we will see in the next section.

## Alternative indexing and data selection

If the explanation in the previous section sounded confusing or ambiguous, or if you didn't understand a thing, you don't have to worry.

There is another way to index Pandas DataFrames, which

* allows use of index pairs to access a single element
* has the same order of dimensions as NumPy: first index specifies rows, second columns
* is not ambiguous about implicit or explicit indices

Pandas DataFrames have attributes `loc` and `iloc` that have the above qualities.
You can use `loc` and `iloc` attributes and forget everything about the previous section. Or you can use these attributes
and sometimes use the methods from the previous section as shortcuts if you understand them well.

The difference between `loc` and `iloc` attributes is that the former uses explicit indices and the latter uses the implicit integer indices. Examples of use:

In [20]:
df.loc[1, "Wage"]

1500

In [21]:
df.iloc[-1,-1]             # Right lower corner of the DataFrame

29

In [22]:
df.loc[1, ["Name", "Wage"]]

Name    John
Wage    1500
Name: 1, dtype: object

With `iloc` everything works like with NumPy arrays: indexing, slicing, fancy indexing, masking and their combinations. With `loc` it is the same but now the names in the explicit indices are used for specifying rows and columns. Make sure your understand why the above examples work as they do!

## Summary statistics

The summary statistic methods work in a similar way as their counter parts in NumPy. By default, the aggregation is done over columns.

In [23]:
wh = pd.read_csv("https://raw.githubusercontent.com/csmastersUH/data_analysis_with_python_2020/master/kumpula-weather-2017.csv")

In [24]:
wh2 = wh.drop(["Year", "m", "d"], axis=1)  # taking averages over these is not very interesting
wh2.mean()

Precipitation amount (mm)    1.966301
Snow depth (cm)              0.966480
Air temperature (degC)       6.527123
dtype: float64

The `describe` method of the `DataFrame` object gives different summary statistics for each (numeric) column. The result is a DataFrame. This method gives a good overview of the data, and is typically used in the exploratory data analysis phase.

In [25]:
wh.describe()

,Year,m,d,Precipitation amount (mm),Snow depth (cm),Air temperature (degC)
count,365.0,365.000000,365.000000,365.000000,358.000000,365.000000
mean,2017.0,6.526027,15.720548,1.966301,0.966480,6.527123
std,0.0,3.452584,8.808321,4.858423,3.717472,7.183934
min,2017.0,1.000000,1.000000,-1.000000,-1.000000,-17.800000
25%,2017.0,4.000000,8.000000,-1.000000,-1.000000,1.200000
50%,2017.0,7.000000,16.000000,0.200000,-1.000000,4.800000
75%,2017.0,10.000000,23.000000,2.700000,0.000000,12.900000
max,2017.0,12.000000,31.000000,35.000000,15.000000,19.600000


## Missing data

You may have noticed something strange in the output of the `describe` method. First, the minimum value in both precipitation and snow depth fields is -1. The special value -1 means that on that day there was absolutely no snow or rain, whereas the value 0 might indicate that the value was close to zero. Secondly, the snow depth column has count 358, whereas the other columns have count 365, one measurement/value for each day of the year. How is this possible? Every field in a DataFrame should have the same number of rows. Let's use the `unique` method of the Series object to find out, which different values are used in this column:

In [26]:
wh["Snow depth (cm)"].unique()

array([ -1.,   7.,  13.,  10.,  12.,   9.,   8.,   5.,   6.,   4.,   3.,
        15.,  14.,   2.,  nan,   0.])

The `float` type allows a special value `nan` (Not A Number), in addition to normal floating point numbers. This value can represent the result from an illegal operation. For example, the operation 0/0 can either cause an exception to occur or just silently produce a `nan`. In Pandas `nan` can be used to represent a missing value. In the weather DataFrame the `nan` value tells us that the measurement from that day is not available, possibly due to a broken measuring instrument or some other problem.

Note that only float types allow the `nan` value (in Python, NumPy or Pandas). So, if we try to create an integer series with missing values, its dtype gets promoted to `float`:

In [27]:
pd.Series([1,3,2])

0    1
1    3
2    2
dtype: int64

In [28]:
pd.Series([1,3,2, np.nan])

0    1.0
1    3.0
2    2.0
3    NaN
dtype: float64

For non-numeric types the special value `None` is used to denote a missing value, and the dtype is promoted to `object`.

In [29]:
pd.Series(["jack", "joe", None])

0    jack
1     joe
2    None
dtype: object

Pandas excludes the missing values from the summary statistics, like we saw in the previous section. Pandas also provides some functions to handle missing values.

The missing values can be located with the `isnull` method:

In [ ]:
wh.isnull()      # returns a boolean mask DataFrame

This is not very useful as we cannot directly use the mask to index the DataFrame. We can, however, combine it with the `any` method to find out all the rows that contain at least one missing value:

In [ ]:
wh[wh.isnull().any(axis=1)]

The `notnull` method works conversively to the `isnull` method.

The `dropna` method of a DataFrame drops columns or rows that contain missing values from the DataFrame, depending on the `axis` parameter.

In [32]:
wh.dropna().shape   # Default axis is 0

(358, 8)

In [33]:
wh.dropna(axis=1).shape # Drops the columns containing missing values

(365, 7)

The `how` and `thresh` parameters of the `dropna` method allow one to specify how many values need to be missing in order for the row/column to be dropped.

The `fillna` method allows to fill the missing values with some constant or interpolated values. The `method` parameter can be:

* `None`: use the given positional parameter as the constant to fill missing values with
* `ffill`: use the previous value to fill the current value
* `bfill`: use the next value to fill the current value

For example, for the weather data we could use forward fill

In [34]:
wh = wh.fillna(method='ffill')
wh[wh.isnull().any(axis=1)]

,Year,m,d,Time,Time zone,Precipitation amount (mm),Snow depth (cm),Air temperature (degC)


The `interpolate` method, which we will not cover here, offers more elaborate ways to interpolate the missing values from their neighbouring non-missing values.

## Converting columns from one type to another

There are several ways of converting a column to another type. For converting single columns (a Series) one can use the `pd.to_numeric` function or the `map` method. For converting several columns in one go one can use the `astype` method. We will give a few examples of use of these methods/functions. For more details, look from the Pandas documentation.

In [35]:
pd.Series(["1","2"]).map(int)                           # str -> int

0    1
1    2
dtype: int64

In [36]:
pd.Series([1,2]).map(str)                               # int -> str

0    1
1    2
dtype: object

In [37]:
pd.to_numeric(pd.Series([1,1.0]), downcast="integer")   # object -> int

0    1
1    1
dtype: int8

In [38]:
pd.to_numeric(pd.Series([1,"a"]), errors="coerce")      # conversion error produces Nan

0    1.0
1    NaN
dtype: float64

In [39]:
pd.Series([1,2]).astype(str)                            # works for a single series

0    1
1    2
dtype: object

In [40]:
df = pd.DataFrame({"a": [1,2,3], "b" : [4,5,6], "c" : [7,8,9]})
print(df.dtypes)
print(df)

a    int64
b    int64
c    int64
dtype: object
   a  b  c
0  1  4  7
1  2  5  8
2  3  6  9


In [41]:
df.astype(float)                       # Convert all columns

,a,b,c
0,1.0,4.0,7.0
1,2.0,5.0,8.0
2,3.0,6.0,9.0


In [42]:
df2 = df.astype({"b" : float, "c" : str})    # different types for columns
print(df2.dtypes)
print(df2)

a      int64
b    float64
c     object
dtype: object
   a    b  c
0  1  4.0  7
1  2  5.0  8
2  3  6.0  9


## String processing

If the elements in a column are strings, then the vectorized versions of Python's string processing methods are available. These are accessed through the `str` attribute of a Series or a DataFrame. For example, to capitalize all the strings of a Series, we can use the `str.capitalize` method:

In [43]:
names = pd.Series(["donald", "theresa", "angela", "vladimir"])
names.str.capitalize()

0      Donald
1     Theresa
2      Angela
3    Vladimir
dtype: object

One can find all the available methods by pressing the tab key after the text `names.str.` in a Python prompt. Try it in below cell!

In [44]:
#names.str.

We can split a column or Series into several columns using the `split` method. For example:

In [45]:
full_names = pd.Series(["Donald Trump", "Theresa May", "Angela Merkel", "Vladimir Putin"])
full_names.str.split()

0      [Donald, Trump]
1       [Theresa, May]
2     [Angela, Merkel]
3    [Vladimir, Putin]
dtype: object

This is not exactly what we wanted: now each element is a list. We need to use the `expand` parameter to split into columns:

In [46]:
full_names.str.split(expand=True)

,0,1
0,Donald,Trump
1,Theresa,May
2,Angela,Merkel
3,Vladimir,Putin


## Additional information

We covered subsetting of DataFrames with the indexers `[]`, `.loc[]`, and `.iloc[]` quite concisely.
For a more verbose explanation, look at the [tutorials at Dunder Data](https://medium.com/dunder-data/pandas-tutorials/home). Especially, the problems with chained indexing operators (like `df["a"][1]`) are explained well there (tutorial 4), which we did not cover at all. As a rule of thumb: one should avoid chained indexing combined with assignment! See [Pandas documentation](http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#why-does-assignment-fail-when-using-chained-indexing).

## Summary

* You can create DataFrames in several ways:
     * By reading from a csv file
     * Out out two dimensional NumPy array
     * Out of rows
     * Out of columns
* You know how to access rows, columns and individual elements of DataFrames
* You can use the `describe` method to get a quick overview of a DataFrame
* You know how missing values are represented in Series and DataFrames, and you know how to manipulate them
* There are similarities between Python's string methods and the vectorized forms of string operations in Series and DataFrames
* You can do complicated text processing with the `str.replace` method combined with regular expressions
* The powerful `where` method is the vectorized form of Python's `if-else` construct
* We remember that with NumPy arrays we preferred vectorized operations instead of, for instance, `for` loops. Same goes with Pandas. It may first feel that things are easier to achieve with loops, but after a while vectorized operations will feel natural.